In [18]:
import pandas as pd
import numpy as np
import math

In [26]:
df = pd.read_csv("../output/weather_csv/Gangneung_weather.csv")
df.head()

,Unnamed: 0,날짜,월,일,평균기온,평균습도,평균풍속,일조시간,풍속계높이
0,0,2007-01-01,1,1,6.0,42.0,2.5,5.0,10
1,1,2007-01-02,1,2,5.6,73.8,2.6,1.0,10
2,2,2007-01-03,1,3,4.4,77.3,2.1,0.0,10
3,3,2007-01-04,1,4,4.8,71.3,2.9,3.5,10
4,4,2007-01-05,1,5,4.7,71.6,2.4,0.0,10


In [20]:
df['북위'] = 37.54
df['해발고도'] = 28

In [21]:
df['u_2'] = df['평균풍속'] * 4.87 / np.log(67.8 * df['풍속계높이'] - 5.42)
df['P'] = 101.3 * ((293 - 0.0065 * df['해발고도']) / 293) ** 5.26
df['Δ'] = df['평균기온'].apply(lambda x:4098 * (0.6108 * np.exp((17.27 * x) / (x + 237.3))) / (x + 237.3) ** 2)
df['gamma'] = 0.665 * 10 ** (-3) * df['P']
# P
df['A'] = 1 + 0.34 * df['u_2']
# Q
df['B'] = df['Δ'] / (df['Δ'] + df['gamma'] * df['A'])
# R
df['C'] = df['gamma'] / (df['Δ'] + df['gamma'] * df['A'])
# S
df['D'] = 900 / (df['평균기온'] + 273) * df['u_2']
df['e_s'] = df['평균기온'].apply(lambda x: 0.6108 * np.exp((17.27 * x) / (x + 237.3)))
df['e_a'] = df['평균습도'] / 100 * df['e_s']
df['e_s-e_a'] = df['e_s'] - df['e_a']
df['doi'] = pd.to_datetime(df['날짜']).dt.strftime('%j')
df['doi'] = df['doi'].apply(pd.to_numeric)
df['dr'] = df['doi'].apply(lambda x: 1 + 0.033 * np.cos(2 * 3.141592 / 365 * x))
df['δ'] = df['doi'].apply(lambda x: 0.409 * np.sin(2 * 3.141592 / 365 * x - 1.39))
df['Φ'] = df['북위'] * math.pi / 180
df['ω_s'] = np.arccos(-df['Φ'].apply(lambda x: np.tan(x))*df['δ'].apply(lambda x: np.tan(x)))


df['Ra'] = 24 * 60 / math.pi * 0.082 * df['dr'] *\
           (df['ω_s']*df['δ'].apply(lambda x: math.sin(x))*
            df['Φ'].apply(lambda x: math.sin(x))+
            df['Φ'].apply(lambda x: math.cos(x))*
            df['δ'].apply(lambda x: math.cos(x))*
            df['ω_s'].apply(lambda x: math.sin(x)))
df['N'] = 24/math.pi*df['ω_s']
df['Rs'] = (0.25+0.5*df['일조시간']/df['N'])*df['Ra']
df['Rso'] = (0.75 + 2 * 10 ** (-5) * df['해발고도']) * df['Ra']
df['Rs/Rso'] = df['Rs'] / df['Rso']
df['R_ns'] = 0.77 * df['Rs']
df['R_nl'] = 4.903*10**(-9)*(df['평균기온']+273.16)**4*(0.34-0.14*df['e_a']**(0.5))*(1.35*df['Rs/Rso']-0.35)
df['Rn'] = df['R_ns'] - df['R_nl']

In [22]:
df

,Unnamed: 0,날짜,월,일,평균기온,평균습도,평균풍속,일조시간,풍속계높이,북위,...,Φ,ω_s,Ra,N,Rs,Rso,Rs/Rso,R_ns,R_nl,Rn
0,0,2007-01-01,1,1,6.0,42.0,2.5,5.0,10,37.54,...,0.655197,1.238935,15.356465,9.464766,7.895335,11.525949,0.685005,6.079408,4.317299,1.762109
1,1,2007-01-02,1,2,5.6,73.8,2.6,1.0,10,37.54,...,0.655197,1.240319,15.411643,9.475338,4.666161,11.567362,0.403390,3.592944,1.297883,2.295061
2,2,2007-01-03,1,3,4.4,77.3,2.1,0.0,10,37.54,...,0.655197,1.241813,15.471234,9.486756,3.867809,11.612090,0.333085,2.978213,0.659584,2.318629
3,3,2007-01-04,1,4,4.8,71.3,2.9,3.5,10,37.54,...,0.655197,1.243418,15.535225,9.499012,6.745856,11.660119,0.578541,5.194309,2.906052,2.288257
4,4,2007-01-05,1,5,4.7,71.6,2.4,0.0,10,37.54,...,0.655197,1.245131,15.603600,9.512099,3.900900,11.711438,0.333085,3.003693,0.671427,2.332266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109,361,2020-12-27,12,27,8.4,43.4,2.0,4.0,10,37.54,...,0.655197,1.234526,15.180152,9.431084,7.014212,11.393615,0.615627,5.400943,3.604778,1.796165
5110,362,2020-12-28,12,28,8.7,58.8,2.2,8.1,10,37.54,...,0.655197,1.235458,15.217547,9.438201,10.334346,11.421682,0.904801,7.957446,6.098458,1.858988
5111,363,2020-12-29,12,29,5.4,59.0,2.5,2.3,10,37.54,...,0.655197,1.236503,15.259404,9.446191,5.672564,11.453099,0.495286,4.367875,2.240222,2.127653
5112,364,2020-12-30,12,30,-5.3,35.0,4.4,6.5,10,37.54,...,0.655197,1.237663,15.305713,9.455048,9.087487,11.487856,0.791052,6.997365,5.197994,1.799372
